In [ ]:
WITH InitialSales AS (
    SELECT  
    a.cIDCompany Company_DwID
    ,a.cIDInvoice Invoice_DwID
    ,a.cSerie Serie
    ,a.cForm Form
    ,a.cIDCustomer Customer_DwID 
    ,a.cIDBranchInvoice BranchDwID
    ,a.cIDTerritory Territory_DwID
    ,a.dEmission Emission
    ,a.xChannelDistribution ChannelDistribution 
    ,a.xInvoiceCancelReason InvoiceCancelReason
    ,b.cidOrder Order_DwID
    ,a.xInvoiceType InvoiceType 
    ,e.cIDProduct Product_DwID
    ,e.nAmount Amount
    ,e.nUnitValue UnitValue
    ,e.nTotalValue TotalValue
    ,e.MC1LastUpdate
    ,e.nFreegoods Freegoods
    ,g.nSeq Seq
    ,g.nValue Value
    ,g.cIDPricing Pricing_DwID
    ,g.cIDStep Step_DwID
    ,f.xIDReasonReturnStock ReasonReturnStock_DwID
    ,h.lCalculated Calculated
    ,h.cIDPricingFormationRule PricingFormationRule_DwID
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_Invoice a
    --################# order invoice #########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_OrderInvoice 
    ) b 
    on a.cIDCompany = b.cIDCompany
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cform
    and a.cSerie = b.cSerie
    and a.cIDBranchInvoice =b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    and b.mc1enabled = 1
    --################## invoice item #####################
    inner join (select *
    FROM CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceItem
    ) e 
    on e.cIDCompany = a.cIDCompany 
    and e.cIDInvoice = a.cIDInvoice 
    and e.cForm = a.cform
    and e.cSerie = a.cSerie
    AND e.cIDBranchInvoice=a.cIDBranchInvoice 
    and e.mc1enabled = 1
    --################ invoice item detail #######################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceItemDetail 
    ) g
    on g.cIDCompany = e.cIDCompany
    and g.cIDInvoice = e.cIDInvoice
    and g.cForm = e.cform
    and g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    and g.nSeq = e.nSeq --igualando pasos 
    and g.cidProduct = e.cidProduct --igualando los productos de factura y order
    and g.mc1enabled = 1
    and g.cIDStep in ('00100','00130','00160','00200','00250','00300','00400','00450','00500','00600','00700','00800','00900','01400','00150') --steps para llegar a la venta neta 
    --################### MC1_MC_OrderExt ########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_OrderExt
    ) f 
    on f.cIDOrder = b.cIDOrder
    and f.cIDCompany = b.cIDCompany
    and f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
    FROM CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceRequestEmites
    ) m
    on a.cIDCompany = m.cIDCompany
    and a.cIDInvoice = m.cIDInvoice
    and a.cForm = m.cform
    and a.cSerie = m.cSerie
    and a.cIDBranchInvoice =m.cIDBranchInvoice
    and m.mc1enabled = 1
    and UPPER(m.xInvoiceRequestStatus)= 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    inner join (
    select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_PricingFormationRule
    ) h
    on g.cIDStep = h.cIDStep
    and g.cIDPricing = h.cIDPricing
    and g.cIDCompany = h.cIDCompany
    and g.mc1enabled = 1
    and h.mc1enabled = 1
    and upper(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED' --se repite el paso 00100 y parece aumentar el valor 
    where a.xInvoiceType not in ('WTM002','WTM005','ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    and format(a.dEmission , 'yyyy-MM-dd') between '2024-11-13' and '2024-11-14' --informacion de un dia  
),
ValidOrders AS(
    SELECT distinct b.cIDOrder, b.cIDProduct,a.cIDBranchinvoice 
	from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_OrderInvoice a
	inner join CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_OrderItem b 
	on a.cIDOrder = b.cIDOrder 
	and a.cIDCompany = b.cIDCompany 
	and a.cIDBranchInvoice = b.cIDBranchinvoice
	where 1=1
	and a.mc1Enabled = 1
	and b.mc1Enabled = 1
	and format(a.mc1lastupdate , 'yyyy-MM-dd') between '2024-10-14' and '2024-11-14' --informacion de un dia 
),
ValidSales AS (
    SELECT 
        a.* 
    FROM InitialSales a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct 
        AND b.cIDOrder is not null
),
FirstDeduplicates AS(
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.BranchDwID, a.ChannelDistribution, 
            a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.FreeGoods, a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType,
            a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID ORDER BY a.mc1LastUpdate DESC, a.ReasonReturnStock_DwID DESC) as rn
    FROM ValidSales a
    ) sub
    WHERE rn = 1
),
Transformations AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN a.Value ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00400' THEN a.Value ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00200' THEN a.Value ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN a.Value ELSE 0 END AS Escalafc_ML,
        CASE 
            WHEN a.InvoiceType = 'ZBON' AND a.Step_DwID = '00800' THEN a.Value
            WHEN a.Step_DwID = '00600' AND a.Freegoods = 0 THEN a.Value
            WHEN a.Step_DwID = '00100' AND a.Freegoods = 1 THEN a.Value 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '00900' THEN a.Value ELSE 0 END AS Impuesto1,
        0 as Impuesto2,
        CASE WHEN a.Step_DwID = '01400' THEN a.Value ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicates a
),
GroupedSales AS (
    SELECT
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM Transformations
    GROUP BY
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue
),
GroupedSales2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM GroupedSales
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue
),
InvoiceItemDetailJoin AS (
    SELECT 
    a.cIDCompany Company_DwID,
    a.cIDBranchInvoice BranchInvoice_DwID,
    a.cSerie Serie,
    a.cIDInvoice Invoice_DwID,
    a.cForm Form,
    a.cIDProduct Product_DwID,
    FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm') AS MC1LastUpdate,
    c.nfreegoods FreeGoods,
    a.nValue AS Value2
    --, SUM(a.Value) AS Value2, COUNT(*) AS no_registros
FROM 
    CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceItemDetail a
INNER JOIN 
    (
        SELECT * 
        FROM CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_Invoice 
        WHERE MC1Enabled = 1
          AND FORMAT(DEmission, 'yyyy-MM-dd') BETWEEN '2024-11-13' AND '2024-11-14' -- información de un día
          AND xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR')
    ) b ON a.cIDBranchInvoice = b.cIDBranchInvoice
       AND a.cSerie = b.cSerie
       AND a.cIDInvoice = b.cIDInvoice
       AND a.cForm = b.cForm
INNER JOIN 
    (
        SELECT * 
        FROM CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_Invoiceitem
        WHERE MC1Enabled = 1
    ) c ON a.cIDBranchInvoice = c.cIDBranchInvoice
       AND a.cSerie = c.cSerie
       AND a.cIDInvoice = c.cIDInvoice
       AND a.cForm = c.cForm
       AND a.cIDProduct = c.cIDProduct
       AND a.nSeq = c.nSeq
WHERE 
    a.cIDStep = '00800'
    AND a.MC1Enabled = 1
GROUP BY 
    a.cIDCompany,
    a.cIDBranchInvoice,
    a.cSerie,
    a.cIDInvoice,
    a.cForm,
    a.cIDProduct,
    FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm'),
    c.nfreegoods,
    a.nValue
),
InvoiceItemDetailDeduplicated AS (
    SELECT DISTINCT 
        *
    FROM InvoiceItemDetailJoin
),
InvoiceDetailGenerated AS (
    SELECT 
        Company_DwID,
        BranchInvoice_DwID,
        Serie,
        Invoice_DwID,
        Form,
        Product_DwID,
        MC1LastUpdate,
        FreeGoods,
        CASE WHEN FreeGoods = 1 THEN 0 ELSE Value2 END AS Value2
    FROM InvoiceItemDetailDeduplicated
),
InvoiceDetailGrouped AS (
    SELECT
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie,
        SUM(Value2) AS facdet
    FROM InvoiceDetailGenerated
    GROUP BY 
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie
),
FinalJoin AS (
    SELECT 
        a.*,
        b.facdet
    FROM GroupedSales2 a
    INNER JOIN InvoiceDetailGrouped b ON
        a.Company_DwID = b.Company_DwID AND
        a.BranchDwID = b.BranchInvoice_DwID AND
        a.Product_DwID = b.Product_DwID AND
        a.Serie = b.Serie AND
        a.Form = b.Form AND
        a.Invoice_DwID = b.Invoice_DwID
),
AddColumns AS (
    SELECT 
        a.BranchDwID codigo_agencia,
        a.ChannelDistribution,
        CASE WHEN a.Company_DwID='PY' THEN 'PARAGUAY' ELSE 'REV' END as pais,
        a.Customer_DwID codigo_cliente,
        CONVERT(VARCHAR(10), CAST(a.Emission AS DATE), 120) fecha,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType AG19ID,
        a.Order_DwID,
        a.Product_DwID codigo_producto,
        a.ReasonReturnStock_DwID,
        a.Serie,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') as codigo_ruta,
        a.UnitValue,
        a.Valor_Bruto,
        a.Facdsl_ML,
        a.Promocion_ML,
        a.Descuento_ML,
        a.Amount,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.Escalafc_ML,
        CONCAT(a.Serie,'-',a.Invoice_DwID) as numero,
        a.ChannelDistribution as cod_linea_ruta,
        'FALSE' as FACCON,
        0 as FACDSL,
        null as dif_precio,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN 0 - a.facdet
            ELSE a.facdet
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN a.ReasonReturnStock_DwID
            WHEN NOT (a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX') THEN a.InvoiceCancelReason
            ELSE 'REVISAR'
        END AS Id_Motivo
    FROM FinalJoin a
),
Subchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, a.cIDCustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_CustomerDivisionSector b 
        ON a.cidcustomer = b.cidcustomer
        WHERE b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
JoinWithSubchannel AS (
    SELECT
        a.*,
        b.xSubChannelDistribution
    FROM AddColumns a
    LEFT JOIN  Subchannel b
    ON a.codigo_cliente = b.cIDCustomer
    AND a.codigo_agencia = b.cidbranchinvoice
),
ModifyColumns AS (
    SELECT 
        a.codigo_agencia,
        a.codigo_cliente CLICOD,
        CONVERT(VARCHAR(10), a.fecha, 120) Fecha_vta,
        a.Invoice_DwID no_factura,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.UnitValue precio,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Facdsl_ML
            ELSE a.Facdsl_ML END AS FACDSL$ ,
         CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Promocion_ML
            ELSE a.Promocion_ML END AS promocion,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Descuento_ML
            ELSE a.Descuento_ML END AS descuento,
        a.Amount cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZBON' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Escalafc_ML
            ELSE a.Escalafc_ML END AS Escalafc_ML,
        a.numero,
        a.FACCON,
        a.FACDSL,
        a.facdet FACDET,
        CASE  
    		WHEN a.cod_linea_ruta <> a.xSubChannelDistribution AND a.xSubChannelDistribution IS NOT NULL THEN a.xSubChannelDistribution ELSE a.cod_linea_ruta 
		END AS cod_linea_ruta,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            ELSE a.Id_Motivo END as Id_Motivo
    FROM JoinWithSubchannel a 
),
ReturnReason AS (
    SELECT 
        CASE
            WHEN cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled,
        Insert_date_local
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_GeneralDescription
    Where cDomainType = 'xInvoiceCancelReason'
    AND lEnabled = 1
),
Motivos AS(
    SELECT 
        CASE
            WHEN cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled,
        Insert_date_local
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_GeneralDescription
    Where cDomainType = 'xIDReasonReturnStock'
    AND lEnabled = 1
    AND UPPER(cDescription) <> 'SOBRESTOCK EN TIENDA'
),
QueryMC1Chile as (
    SELECT 
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.no_factura,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.FACDSL$,
        a.promocion,
        a.descuento,
        a.cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.Escalafc_ML,
        a.numero,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.cod_linea_ruta,
        a.Id_Motivo,
        CASE
            WHEN b.Description is null AND c.Description <> ''
            THEN c.Description
            ELSE b.Description
        END AS factura_cliente,
        0 AS dif_precio
    FROM ModifyColumns a
    LEFT JOIN ReturnReason b
    ON a.InvoiceCancelReason = b.cIDDomainType
    LEFT JOIN Motivos c
    ON a.Id_Motivo = c.cIDDomainType
    Where a.codigo_ruta NOT LIKE '%0A%'
    AND a.Fecha_vta  >= '2022-11-20'
),
QueryMC1ChileCast AS (
    SELECT 
        CAST(a.CLICOD AS VARCHAR(250)) AS CLICOD,
        CAST(a.Fecha_vta AS DATE) AS Fecha_vta,
        CAST(a.no_factura AS VARCHAR(250)) AS no_factura,
        CAST(a.codigo_agencia AS VARCHAR(250)) AS codigo_agencia,
        CAST(a.AG19ID AS VARCHAR(250)) AS AG19ID,
        CAST(a.codigo_producto AS VARCHAR(250)) AS codigo_producto,
        CAST(a.codigo_ruta AS VARCHAR(250)) AS codigo_ruta,
        CAST(a.precio AS DECIMAL(10, 2)) AS precio,
        CAST(a.dif_precio AS DECIMAL(10, 2)) AS dif_precio,
        CAST(a.FACDSL$ AS DECIMAL(10, 2)) AS FACDSL$,
        CAST(a.promocion AS VARCHAR(50)) AS promocion,
        CAST(a.descuento AS DECIMAL(10, 2)) AS descuento,
        CAST(a.cantidad AS INT) AS cantidad,
        CAST(a.Impuesto1 AS DECIMAL(10, 2)) AS Impuesto1,
        CAST(a.Venta_Impuestos AS DECIMAL(10, 2)) AS Venta_Impuestos,
        CAST(a.Impuesto2 AS DECIMAL(10, 2)) AS Impuesto2,
        CAST(a.Escalafc_ML AS DECIMAL(10, 2)) AS Escalafc_ML,
        CAST(a.numero AS VARCHAR(50)) AS numero,
        CAST(a.cod_linea_ruta AS VARCHAR(50)) AS cod_linea_ruta,
        CAST(a.FACCON AS VARCHAR(50)) AS FACCON,
        CAST(a.FACDSL AS VARCHAR(50)) AS FACDSL,
        CAST(a.FACDET AS DECIMAL(10, 2)) AS FACDET,
        CAST(a.Id_Motivo AS VARCHAR(250)) AS Id_Motivo,
        CAST(a.factura_cliente AS VARCHAR(50)) AS factura_cliente
    FROM QueryMC1Chile a
),
--------------------------------RECHAZOS--------------------------------
InitialRechazos AS (
    SELECT  
    a.cIDCompany Company_DwID
    ,a.cIDInvoice Invoice_DwID
    ,a.cSerie Serie
    ,a.cForm Form
    ,a.cIDCustomer Customer_DwID 
    ,a.cIDBranchInvoice BranchDwID
    ,a.cIDTerritory Territory_DwID
    ,a.dEmission Emission
    ,a.xChannelDistribution ChannelDistribution 
    ,a.xInvoiceCancelReason InvoiceCancelReason
    ,b.cidOrder Order_DwID
    ,a.xInvoiceType InvoiceType 
    ,e.cIDProduct Product_DwID
    ,e.nAmount Amount
    ,e.nUnitValue UnitValue
    ,e.nTotalValue TotalValue
    ,e.MC1LastUpdate
    ,e.nFreegoods Freegoods
    ,g.nSeq Seq
    ,g.nValue Value
    ,g.cIDPricing Pricing_DwID
    ,g.cIDStep Step_DwID
    ,f.xIDReasonReturnStock ReasonReturnStock_DwID
    ,h.lCalculated Calculated
    ,h.cIDPricingFormationRule PricingFormationRule_DwID
    ,CASE 
    WHEN i.lPartialDelivery = '1' and l.nAmount!=0 THEN 'RECHAZO PARCIAL'
        ELSE 'RECHAZO TOTAL'  end Tipo_Rechazo
    ,CASE 
    WHEN i.lPartialDelivery = '1' and l.nAmount!=0 THEN '1'
        ELSE '2'  end Tipo_Rechazo_Id
    ,k.cIDInvoice InvoiceFac_DwID
    ,k.cSerie SerieFac
    ,l.nAmount AmountFac
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_Invoice a
    --################# order invoice #########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_OrderInvoice 
    ) b 
    on a.cIDCompany = b.cIDCompany
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cform
    and a.cSerie = b.cSerie
    and a.cIDBranchInvoice =b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    and b.mc1enabled = 1
    --################## invoice item #####################
    inner join (select *
    FROM CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceItem
    ) e 
    on e.cIDCompany = a.cIDCompany 
    and e.cIDInvoice = a.cIDInvoice 
    and e.cForm = a.cform
    and e.cSerie = a.cSerie
    AND e.cIDBranchInvoice=a.cIDBranchInvoice 
    and e.mc1enabled = 1
    --################ invoice item detail #######################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceItemDetail 
    ) g
    on g.cIDCompany = e.cIDCompany
    and g.cIDInvoice = e.cIDInvoice
    and g.cForm = e.cform
    and g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    and g.nSeq = e.nSeq --igualando pasos 
    and g.cidProduct = e.cidProduct --igualando los productos de factura y order
    and g.mc1enabled = 1
    and g.cIDStep in ('00100','00200','00400','00600','00700','00800','00900','01100','01200','01300') --steps para llegar a la venta neta 
    --################### MC1_MC_OrderExt ########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_OrderExt
    ) f 
    on f.cIDOrder = b.cIDOrder
    and f.cIDCompany = b.cIDCompany
    and f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
    FROM CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceRequestEmites
    ) m
    on a.cIDCompany = m.cIDCompany
    and a.cIDInvoice = m.cIDInvoice
    and a.cForm = m.cform
    and a.cSerie = m.cSerie
    and a.cIDBranchInvoice =m.cIDBranchInvoice
    and m.mc1enabled = 1
    and UPPER(m.xInvoiceRequestStatus)= 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    inner join (
    select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_PricingFormationRule
    ) h
    on g.cIDStep = h.cIDStep
    and g.cIDPricing = h.cIDPricing
    and g.cIDCompany = h.cIDCompany
    and g.mc1enabled = 1
    and h.mc1enabled = 1
    and upper(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED' --se repite el paso 00100 y parece aumentar el valor 
    --############# invoiceext para partial ############
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceExt 
    ) i 
    on a.cIDCompany = i.cIDCompany
    and a.cIDInvoice = i.cIDInvoice
    and a.cForm = i.cform
    and a.cSerie = i.cSerie
    and a.cIDBranchInvoice =i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    ---#################### order para encontrar orden de factura ##########
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_Order
    ) j
    on f.cIDOrderRef = j.cIDOrder --trayendo solo lo que genero otro docto, por rechazo parcial
    and f.cIDCompany = j.cIDCompany
    and f.cidcustomer = j.cidcustomer
    and j.mc1enabled = 1
    --################# order invoice para factura parcial #########################
    left join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_OrderInvoice 
    ) k 
    on j.cIDOrder = k.cIDOrder --trayendo solo lo que genero otro docto, por rechazo parcial
    and j.cIDCompany = k.cIDCompany
    AND k.mc1Enabled = 1
    --################## invoice item por producto #####################
    left join (select *
    FROM CBRTM3XPOLCD22_LASC1.GRO_ID_RTM.VW_MC1_InvoiceItem
    ) l
    on l.cIDCompany = k.cIDCompany 
    and l.cIDInvoice = k.cIDInvoice 
    and l.cForm = k.cform
    and l.cSerie = k.cSerie
    AND l.cIDBranchInvoice=k.cIDBranchInvoice 
    and l.mc1enabled = 1
    and l.cidProduct = e.cidProduct --igualando los productos de factura y remito
    where a.xInvoiceType in ('ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    and format(a.dEmission , 'yyyy-MM-dd') between '2024-11-13'  and '2024-11-14' --informacion de un dia  
),
JoinWithValidOrders AS (
    SELECT 
        a.* 
    FROM InitialRechazos a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct AND
        a.BranchDwID = b.cIDBranchInvoice 
        AND b.cIDOrder is not null   
),
FirstDeduplicatesRechazos AS(
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.BranchDwID, a.ChannelDistribution, 
            a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.FreeGoods, a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType,
            a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID ORDER BY a.mc1LastUpdate DESC, a.ReasonReturnStock_DwID DESC) as rn
    FROM JoinWithValidOrders a
    ) sub
    WHERE rn = 1
),
TransformationsRechazos AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN a.Value ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00400' THEN a.Value ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00600' THEN a.Value ELSE 0 END AS Descuento_ML,
        --CASE WHEN a.Step_DwID = '00150' THEN a.Value ELSE 0 END AS Escalafc_ML,
        CASE 
            WHEN a.Step_DwID = '00200' AND a.Freegoods = 0 THEN a.Value
            WHEN a.Step_DwID = '00100' AND a.Freegoods = 1 THEN a.Value 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '01300' THEN a.Value ELSE 0 END AS Impuesto1,
        CASE WHEN a.Step_DwID = '01100' THEN a.Value ELSE 0 END AS Impuesto2,
        CASE WHEN a.Step_DwID = '01400' THEN a.Value ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicatesRechazos a
),
GroupedSalesRechazos AS (
    SELECT
        Amount,
        AmountFac,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2
    FROM TransformationsRechazos
    GROUP BY
        Amount,
        AmountFac,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue
),
GroupedSalesRechazos2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(AmountFac) AS AmountFac
    FROM GroupedSalesRechazos
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceFac_DwID,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        SerieFac,
        Territory_DwID,
        Tipo_Rechazo,
        Tipo_Rechazo_Id,
        UnitValue
),
AddColumnsRechazos AS (
    SELECT 
        a.BranchDwID codigo_agencia,
        a.ChannelDistribution,
        CASE WHEN a.Company_DwID='PY' THEN 'PARAGUAY' ELSE 'REV' END as pais,
        a.Customer_DwID codigo_cliente,
        CONVERT(VARCHAR(10), CAST(a.Emission AS DATE), 120) fecha,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceFac_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType AG19ID,
        a.Order_DwID,
        a.Product_DwID codigo_producto,
        a.ReasonReturnStock_DwID,
        a.Serie,
        a.SerieFac,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') as codigo_ruta,
        a.UnitValue,
        a.Valor_Bruto,
        a.Facdsl_ML,
        a.Promocion_ML,
        a.Descuento_ML,
        a.Amount,
        a.AmountFac,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.Tipo_Rechazo,
        a.Tipo_Rechazo_Id,
        CONCAT(a.Serie,'-',a.Invoice_DwID) as numero,
        CONCAT(a.SerieFac,'-',a.InvoiceFac_DwID) as numero_fac,
        a.ChannelDistribution as cod_linea_ruta,
        'FALSE' as FACCON,
        0 as FACDSL,
        null as dif_precio,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' OR a.InvoiceType = 'ZOGRD' OR a.InvoiceType = 'ZOGR' THEN 0
            ELSE 0
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' OR a.InvoiceType = 'ZOGRD' OR a.InvoiceType = 'ZOGR' 
            THEN CONCAT(a.Tipo_Rechazo_Id,'-',a.InvoiceCancelReason)
            ELSE 'REVISAR'
        END AS Id_Motivo
    FROM GroupedSalesRechazos2 a
),
JoinRechazosWithSubchannel AS (
    SELECT
        a.*,
        b.xSubChannelDistribution
    FROM AddColumnsRechazos a
    LEFT JOIN  Subchannel b
    ON a.codigo_cliente = b.cIDCustomer
    AND a.codigo_agencia = b.cidbranchinvoice
),
ModifyColumnsRechazos AS (
    SELECT 
        a.codigo_agencia,
        a.codigo_cliente CLICOD,
        CONVERT(VARCHAR(10), a.fecha, 120) Fecha_vta,
        a.Invoice_DwID no_factura,
        a.InvoiceFac_DwID,
        a.SerieFac,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.UnitValue precio,
        a.dif_precio,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZBON' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOGRD' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOGR'
            THEN 0-a.Facdsl_ML
            ELSE a.Facdsl_ML END AS FACDSL$ ,
         CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZBON' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOGRD' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOGR'
            THEN 0-a.Promocion_ML
            ELSE a.Promocion_ML END AS promocion,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZBON' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOGRD' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOGR'
            THEN 0-a.Descuento_ML
            ELSE a.Descuento_ML END AS descuento,
        a.Amount cantidad,
        a.AmountFac,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.numero,
        a.numero_fac,
        a.Tipo_Rechazo,
        a.Tipo_Rechazo_Id,
        a.FACCON,
        a.FACDSL,
        a.facdet FACDET,
        CASE  
    		WHEN a.cod_linea_ruta <> a.xSubChannelDistribution AND a.xSubChannelDistribution IS NOT NULL THEN a.xSubChannelDistribution ELSE a.cod_linea_ruta 
		END AS cod_linea_ruta,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            ELSE a.Id_Motivo END as Id_Motivo
    FROM JoinRechazosWithSubchannel a 
),
JoinWithReturnReasonAndMotivos AS (
    SELECT
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.no_factura,
        a.InvoiceFac_DwID,
        a.SerieFac,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.dif_precio,
        a.FACDSL$ ,
        a.promocion,
        a.descuento,
        CASE
            WHEN a.cantidad - isnull(a.AmountFac,0) = 0
            THEN a.cantidad
            ELSE a.cantidad-isnull(a.AmountFac,0)
        END AS cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.numero,
        CASE WHEN a.numero_fac = '-' THEN '' ELSE a.numero_fac END AS numero_fac,
        a.Tipo_Rechazo_Id,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.Id_Motivo,
        a.cod_linea_ruta,
        isnull(a.AmountFac,0) AmountFac,
        CONCAT(a.Tipo_Rechazo, '-', b.Description) AS factura_cliente
    FROM ModifyColumnsRechazos a 
    LEFT JOIN ReturnReason b
    ON a.InvoiceCancelReason = b.cIDDomainType
    LEFT JOIN Motivos c
    ON a.Id_Motivo = c.cIDDomainType
    WHERE b.Description is not null
    AND a.codigo_ruta NOT LIKE '%0A%'
    AND CONCAT(a.Tipo_Rechazo, '-', b.Description) is not null
),
QueryRechazosMC1Chile as (
    SELECT 
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.no_factura,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.dif_precio,
        a.FACDSL$ ,
        a.promocion,
        a.descuento,
        a.cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.numero,
        a.cod_linea_ruta,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.Id_Motivo,
        a.factura_cliente,
        0 as Escalafc_ML
    FROM JoinWithReturnReasonAndMotivos a
    WHERE a.cantidad <> 0
),
QueryRechazosMC1ChileCast AS (
    SELECT 
        CAST(a.CLICOD AS VARCHAR(250)) AS CLICOD,
        CAST(a.Fecha_vta AS DATE) AS Fecha_vta,
        CAST(a.no_factura AS VARCHAR(250)) AS no_factura,
        CAST(a.codigo_agencia AS VARCHAR(250)) AS codigo_agencia,
        CAST(a.AG19ID AS VARCHAR(250)) AS AG19ID,
        CAST(a.codigo_producto AS VARCHAR(250)) AS codigo_producto,
        CAST(a.codigo_ruta AS VARCHAR(250)) AS codigo_ruta,
        CAST(a.precio AS DECIMAL(10, 2)) AS precio,
        CAST(a.dif_precio AS DECIMAL(10, 2)) AS dif_precio,
        CAST(a.FACDSL$ AS DECIMAL(10, 2)) AS FACDSL$,
        CAST(a.promocion AS VARCHAR(50)) AS promocion,
        CAST(a.descuento AS DECIMAL(10, 2)) AS descuento,
        CAST(a.cantidad AS INT) AS cantidad,
        CAST(a.Impuesto1 AS DECIMAL(10, 2)) AS Impuesto1,
        CAST(a.Venta_Impuestos AS DECIMAL(10, 2)) AS Venta_Impuestos,
        CAST(a.Impuesto2 AS DECIMAL(10, 2)) AS Impuesto2,
        CAST(a.Escalafc_ML AS DECIMAL(10, 2)) AS Escalafc_ML,
        CAST(a.numero AS VARCHAR(50)) AS numero,
        CAST(a.cod_linea_ruta AS VARCHAR(50)) AS cod_linea_ruta,
        CAST(a.FACCON AS VARCHAR(50)) AS FACCON,
        CAST(a.FACDSL AS VARCHAR(50)) AS FACDSL,
        CAST(a.FACDET AS DECIMAL(10, 2)) AS FACDET,
        CAST(a.Id_Motivo AS VARCHAR(250)) AS Id_Motivo,
        CAST(a.factura_cliente AS VARCHAR(50)) AS factura_cliente
    FROM QueryRechazosMC1Chile a
),
FinalUnion AS (
SELECT * FROM QueryMC1ChileCast
UNION ALL
SELECT * FROM QueryRechazosMC1ChileCast
)
SELECT * FROM FinalUnion
